# Transformation des données de simulation en données de test

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

In [2]:
original_dir_path = Path.home() / "Documents" / "data" / "Safire_meghatropique" / "simulations" / "icing" / "short_sim"
output_dir_path = Path.cwd() / "test_data"

original_file_paths = list(original_dir_path.glob("*.csv"))[:15]
output_file_path = output_dir_path / "test_data.csv"

In [3]:
def transform_files(
    original_file_paths,
    output_file_path,
    column_mapping,
):
    res_df = pd.DataFrame()
    for fp in original_file_paths:
        flight_name = fp.stem
        df = pd.read_csv(fp)
        transformed_df = pd.DataFrame()
        for new_column_name, old_column_name in column_mapping.items():
            transformed_df[new_column_name] = df[old_column_name]
        transformed_df = transform_altitude(transformed_df)
        transformed_df = transform_mass(transformed_df)
        transformed_df = transform_flaps(transformed_df)
        transformed_df = transform_rad_to_deg(transformed_df)
        transformed_df = transform_kelvin_to_celcius(transformed_df)
        transformed_df = transform_ms_to_kts(transformed_df)
        transformed_df = transform_pascal_to_mbar(transformed_df)
        transformed_df = round_time(transformed_df)
        transformed_df["flight_name"] = flight_name
        res_df = pd.concat([res_df, transformed_df])
    res_df = transform_flight_names(res_df)
    res_df.to_csv(output_file_path, index=False)

def transform_altitude(df):
    df["ZBPIL"] = - df["ZBPIL"]
    return df

def transform_mass(df):
    df["MASS"] = df["MASS"] * 2.2046
    return df

def transform_flaps(df):
    df["DB"] = df["DB"].apply(lambda x: 1 if x > 0.01 else 0)
    return df

def transform_rad_to_deg(df):
    df["ALPHA"] = df["ALPHA"] * 180 / np.pi
    df["BETA"] = df["BETA"] * 180 / np.pi
    df["DM"] = df["DM"] * 180 / np.pi
    df["DN"] = df["DN"] * 180 / np.pi
    df["DSPOIL"] = df["DSPOIL"] * 180 / np.pi
    df["PHI"] = df["PHI"] * 180 / np.pi
    return df

def transform_kelvin_to_celcius(df):
    df["SAT"] = df["SAT"] - 273.15
    return df

def transform_ms_to_kts(df):
    df["VTAS"] = df["VTAS"] * 1.852
    return df

def transform_pascal_to_mbar(df):
    df["PST"] = df["PST"] / 100
    return df

def round_time(df):
    df["Time, s"] = df["Time, s"].round(2)

def transform_flight_names(df):
    unique_values = df["flight_name"].unique()
    value_to_numeric = {value: f"vol {idx}" for idx, value in enumerate(unique_values)}
    df["flight_name"] = df["flight_name"].map(value_to_numeric)
    return df

In [4]:
column_mapping = {
    "Time, s": "Time, s",
    "MASS": "Mass of the aircraft, m, kg",
    "JX": "X load factor",
    "JY": "Y load factor",
    "JZ": "Z load factor",
    "ALPHA": "Incidence angle alpha, rad",
    "BETA": "Sideslip angle beta, rad",
    "PST": "Air Pressure, N/m^2",
    "SAT": "Air Temperature, K",
    "VTAS": "Air speed, m/s",
    "MACH": "Mach number",
    "MLG": "Position of the gear, %",
    "DB": "Position of the flaps, fl, rad",
    "DVOLIG": "Position of the flaps, fl, rad",
    "TRIM": "Stabilator, dSr, rad",
    "DM": "Elevator, dEr, rad, positive: trailing edge down",
    "DN": "Rudder, dRr, rad, positive: trailing edge left",
    "DL": "Aileron, dAr, rad, positive: left trailing edge down",
    "DSPOIL": "Asymmetric Spoiler, dASr, rad",
    "TROTTLE": "Throttle, dT, %",
    "N1": "Throttle, dT, %", # pas d'info sur le N1 donc on le prend identique au throttle
    "ZBPIL": "Negative of c.m. altitude WRT Earth, ze = -h, m",
    "PHI": "Roll angle of body WRT Earth, phir, rad",
}

In [5]:
transform_files(
    original_file_paths,
    output_file_path,
    column_mapping,
)